In [41]:
import pandas as pd
import numpy as np
import datetime
import os

In [42]:
#pd.set_option('display.max_columns', None)
path = os.path.join('..', 'data', 'donors.csv')

donors = pd.read_csv(path, sep = ',')

pd.set_option('display.max_columns', None)

D:\Programas\Anaconda\envs\dmi\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (9,412) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [43]:
donors.columns.tolist()

['Unnamed: 0',
 'ODATEDW',
 'OSOURCE',
 'TCODE',
 'STATE',
 'ZIP',
 'MAILCODE',
 'PVASTATE',
 'DOB',
 'NOEXCH',
 'RECINHSE',
 'RECP3',
 'RECPGVG',
 'RECSWEEP',
 'MDMAUD',
 'DOMAIN',
 'HOMEOWNR',
 'CHILD03',
 'CHILD07',
 'CHILD12',
 'CHILD18',
 'NUMCHLD',
 'INCOME',
 'GENDER',
 'WEALTH1',
 'HIT',
 'MBCRAFT',
 'MBGARDEN',
 'MBBOOKS',
 'MBCOLECT',
 'MAGFAML',
 'MAGFEM',
 'MAGMALE',
 'PUBGARDN',
 'PUBCULIN',
 'PUBHLTH',
 'PUBDOITY',
 'PUBNEWFN',
 'PUBPHOTO',
 'PUBOPP',
 'DATASRCE',
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'WWIIVETS',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'SOLP3',
 'SOLIH',
 'MAJOR',
 'WEALTH2',
 'GEOCODE',
 'COLLECT1',
 'VETERANS',
 'BIBLE',
 'CATLG',
 'HOMEE',
 'PETS',
 'CDPLAY',
 'STEREO',
 'PCOWNERS',
 'PHOTO',
 'CRAFTS',
 'FISHER',
 'GARDENIN',
 'BOATS',
 'WALKER',
 'KIDSTUFF',
 'CARDS',
 'PLATES',
 'LIFESRC',
 'PEPSTRFL',
 'POP901',
 'POP902',
 'POP903',
 'POP90C1',
 'POP90C2',
 'POP90C3',
 'POP90C4',
 'POP90C5',
 'ETH1',
 'ETH2',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'E

# start


In [44]:
#BELO - COHERENCE
#it has more than 50 states bc it includes US territories like the Virgin Islands, Northern Mariana Islands, Guam, etc
donors['STATE'].nunique()

57

In [45]:
#no donors after today's date
donors['ODATEDW'] = pd.to_datetime(donors['ODATEDW'])
(donors['ODATEDW'] > datetime.datetime.now()).sum()

0

In [46]:
donors['NOEXCH'].value_counts()
#drop this! (incoherent, bc the variable is not very relevant and it 
#is not consistent with the metadata's variables explanation -> explicar

#melhor no relatório)

0    61203
0    33882
1      195
1       90
X       35
         7
Name: NOEXCH, dtype: int64

In [47]:
donors['GENDER'].value_counts() 
# A and C are incoherent, we will assume as U: Unknown

F    51277
M    39094
      2957
U     1715
J      365
A        2
C        2
Name: GENDER, dtype: int64

In [48]:
print('< 0: \n', (donors.iloc[:, 41:47] < 0).sum())
print('> 100: \n', (donors.iloc[:, 41:47] > 100).sum())

#no incoherence

< 0: 
 MALEMILI    0
MALEVET     0
VIETVETS    0
WWIIVETS    0
LOCALGOV    0
STATEGOV    0
dtype: int64
> 100: 
 MALEMILI    0
MALEVET     0
VIETVETS    0
WWIIVETS    0
LOCALGOV    0
STATEGOV    0
dtype: int64


In [49]:
donors = donors[donors['AGE902'] != 0]

In [50]:
donors['AGE906'].min()

27

In [51]:
#PROBLEM on AGE902: "Median Age of Adults 18 or Older" but it had a value that was 0... actually, after analyzing the 
#subset of donors that had AGE902 equal to 0, we found out that the vast majority of them had also 0 in basically all
#socio-demographic variables. This leads us to belive taht the missing values on this columns may be represented by a 0 other than
#a Numpy NaN. Anyways, these records have incoherences, and for this reason, they should be removed (799 records!).

#They have donations... but they should be removed anyways. 
#bc it would be a lot of records and diferente variables to predict

#after removing this records other incoherences in other variables were also removed

In [52]:
(donors.iloc[:, 75:358]).min()
#(donors.iloc[:, 75:358]).max()

## percentagens sem incoerencias (tudo entre 0-10), nao sei se precisamos incluir


#  Minimos:
#HHP1/2 == 0 doenst make sense
#HV1/2 a house with 0 value ??
#4/5 rent == 0 ?? 
#RPH's dont make sense to have avg of 0 rooms per housing unit 
#IC1:5 also doesnt make sense we have avg and medians of value 0
#TPE10/11 MEDIAN == 0 ??
#EC1 lowest values == 89 ?? doesnt make any sense 
#MHUC1/2 also minimum value of 0

#most of median minimum values == 0 doesnt make sense.. DROP?????


# Maximos:
# HHP1/2 == 650/700 how is that possible??
# RPH's value are too high 85/90/61/40
## EC1 -- 170 years of median, impossible
#MHUC1/2 max value 21/5 values are too low 


#I think most or all median values are incorrect. 
#Some are impossible and other are possible but dont make sense I a real world scenario
#for that I think they should be removed and use only the percentages for our analysis

POP903     0.0
POP90C1    0.0
POP90C2    0.0
POP90C3    0.0
POP90C4    0.0
          ... 
HC20       0.0
HC21       0.0
MHUC1      0.0
MHUC2      0.0
AC1        0.0
Length: 283, dtype: float64

In [53]:
#PROBLEM on HV1: "Median Home Value in hundreds" has values of 0, and then, the second lowest, 75 (which means 7500)... 
#the values of 0 are incoherences!!? (Belo) (318 records - same on HV2)


In [54]:
#EC1: "Median Years of School Completed by Adults 25+" has minimum 89 and maximum 170... are the values wrong?

#HC1: "Percent Median Length of Residence" is percent and median at the same time? And has values of 0 length! Is it the 
#median time someone has been living in their current home, in years? Very ambiguous:
#HC2: this one has a less ambiguous description but it still says "Percent Median" so is it a percentage or a median? 
#it is confusing

In [55]:
donors['ODATEDW'].head()

0   2009-01-01
1   2014-01-01
2   2010-01-01
3   2007-01-01
4   2006-01-01
Name: ODATEDW, dtype: datetime64[ns]

In [87]:
#now the dates are datetime
donors['FISTDATE'] = pd.to_datetime(donors['FISTDATE'], errors='ignore')

#            FISTDATE:  Date of first gift 
#                cant be before 
#   ODATEDW: Origin Date. Date of donor's first gift to PVA

#                  RIGHT ?!!!!!!!!!!!!!????????????

(donors['FISTDATE'] < donors['ODATEDW']).sum()

## FISTDATE < ODATEDW substituir por ODATEDW - incoerencia

17445

In [57]:
donors['ODATEDW'] = pd.to_datetime(donors['ODATEDW'])
donors['DOB'] = pd.to_datetime(donors['DOB'], errors='coerce')
donors.iloc[:, 350:373] = donors.iloc[:, 350:373].apply(pd.to_datetime, errors='coerce') #from ADATE_2 until ADATE_24
donors['MAXADATE'] = pd.to_datetime(donors['MAXADATE'])
donors.iloc[:, 378:400] = donors.iloc[:, 378:400].apply(pd.to_datetime, errors='coerce') #from RDATE_3 until RDATE_24
donors['MINRDATE'] = pd.to_datetime(donors['MINRDATE'])
donors['MAXRDATE'] = pd.to_datetime(donors['MAXRDATE'])
donors['LASTDATE'] = pd.to_datetime(donors['LASTDATE'])
donors['FISTDATE'] = pd.to_datetime(donors['FISTDATE'], errors='coerce')
donors['NEXTDATE'] = pd.to_datetime(donors['NEXTDATE'], errors='coerce')

###### DOB incoherence

In [58]:
#DOB incoherence
#Assuming we are in 2020 we have donors with less than 18 years (we dont have nothing saying that they must be 18
#so we will keep the variables). What we are going to check is if they have children if they are less than 16

(donors.loc[:, 'DOB'] > datetime.datetime(2020, 1, 1)).sum()
#No DOB after 2020-1-1


0

In [59]:
donors.DOB.isnull().sum()
#24k MV's

23488

In [60]:
(donors.loc[:, 'DOB'] > datetime.datetime(2004, 12, 31)).sum()
#we have 424 donors with less than 16 years

424

In [68]:
donors.loc[(donors['DOB']> datetime.datetime(2004, 12, 31)) & (donors['NUMCHLD'] > 0), ['DOB', 'NUMCHLD', 'GENDER']]
#At least 24 children under 16 yo have 1 or more child... drop??

,DOB,NUMCHLD,GENDER
6892,2010-09-01,1.0,M
7214,2010-09-01,1.0,F
10687,2007-03-01,3.0,M
11591,2010-02-01,1.0,M
19864,2010-03-01,4.0,F
25211,2010-09-01,1.0,M
25335,2010-08-01,2.0,M
25598,2010-05-01,1.0,M
37897,2010-09-01,1.0,F
40851,2010-06-01,1.0,F


##### NUMCHLD INCOHERENCE

In [62]:
donors['CHILD03'].unique()

array([' ', 'M', 'F', 'B'], dtype=object)

In [63]:
donors['CHILD07'].unique()

array([' ', 'M', 'B', 'F'], dtype=object)

In [64]:
donors['CHILD12'].unique()

array([' ', 'F', 'M', 'B'], dtype=object)

In [65]:
donors['CHILD18'].unique()

array([' ', 'M', 'F', 'B'], dtype=object)

In [66]:
#Assuming that when we have: ' ' the donors has no child

(donors.loc[(donors['CHILD03'] == ' ') & (donors['CHILD07'] == ' ') & (donors['CHILD12'] == ' ') & (donors['CHILD18'] == ' '),
           'NUMCHLD'])

0        NaN
2        NaN
3        NaN
4        1.0
5        NaN
        ... 
95406    NaN
95407    NaN
95409    NaN
95410    NaN
95411    NaN
Name: NUMCHLD, Length: 88742, dtype: float64

In [67]:
donors['NUMCHLD'].isnull().sum()

82283

In [79]:
donors.replace({'OSOURCE' : {' ' : np.nan},
'MAILCODE' : {' ' : '0', 'B' : '1'},
'PVASTATE' : {' ' : 'N'}, #N = state is not served by any organization
'RECINHSE' : {' ' : '0', 'X' : '1'},
'RECP3' : {' ' : '0', 'X' : '1'},
'RECPGVG': {' ' : '0', 'X' : '1'},
'RECSWEEP': {' ' : '0', 'X' : '1'},
'HOMEOWNR' : {' ' : '0', 'H' : '1', 'U' : '0'}, #we assume the blank space on this variable says that it is unknown wether the donor has or not a home (so, we joined it with the 0-Unkwown)
'CHILD03' : {' ' : 'N'}, #the ' ' (blank) will be replaced by 'N' (N = None), as we assume the blank space in this variable means the donor has no
'CHILD07' : {' ' : 'N'}, #children in that specific range of ages
'CHILD12': {' ' : 'N'}, #note: more than 90 000 records have None, on each of these 4 variables
'CHILD18': {' ' : 'N'},
'NUMCHLD': {np.nan : 0}, #assuming NaN is zero, bc there is no zero on this variable!?
'GENDER' : {' ' : 'U', 'A' : 'U', 'C' : 'U'}, #Incoherence (Belo!) -> nem que se fale só no relatório!
#values 'A' (2 of them) & 'C' (2) are not described on the metadata, most probably those are errors
#we assume the blank spaces, As and Cs can be assumed to be Unkwown
'SOLP3' : {' ' : 'Default'},
'SOLIH' : {' ' : 'Default'}, #feature engineering: create a binary with -> 0 (can't be mailed), 1 (can be mailed)
'MAJOR' : {' ' : '0', 'X' : '1'},
'GEOCODE2' : {' ' : np.nan}}, inplace = True)

In [70]:
donors['NUMCHLD'].unique() #assuming nan is zero childs

array([nan,  1.,  2.,  3.,  4.,  5.,  6.,  7.])

In [82]:
(donors.loc[(donors['CHILD03'] != 'N') | (donors['CHILD07'] != 'N') | (donors['CHILD12'] != 'N') | (donors['CHILD18'] != 'N')
            | (donors['NUMCHLD'] != 0), 'NUMCHLD'])

1        1.0
4        1.0
6        1.0
10       0.0
12       2.0
        ... 
95373    0.0
95375    1.0
95377    2.0
95404    1.0
95408    1.0
Name: NUMCHLD, Length: 13869, dtype: float64

In [ ]:
#New column CHILD: 1 has children(filhos) 0 cc

